# Task 4
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, DistilBertModel

from torch.optim import AdamW

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
#model_transformer = DistilBertModel.from_pretrained("distilbert-base-uncased")
# Add any other imports you need here

ModuleNotFoundError: No module named 'torch'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Depending on your approach, you might need to adapt the structure of this template or parts not marked by TODOs.
It is not necessary to completely follow this template. Feel free to add more code and delete any parts that are not required.

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#model_transformer=model_transformer.to(DEVICE)

BATCH_SIZE = 16  # TODO: Set the batch size according to both training performance and available memory
NUM_EPOCHS = 2  # TODO: Set the number of epochs
MAX_LEN = 128

train_val = pd.read_csv("train.csv")
test_val = pd.read_csv("test_no_score.csv")

#len(test_val)
#test_val["title"]+". "+test_val["sentence"]
#test_val["title"][0]
#index=1
#cat=train_val.iloc[index]["title"]+". "+train_val.iloc[index]["sentence"]
#inputs = tokenizer(cat, return_tensors="pt")
#outputembedding = model_transformer(**inputs)
#outputembedding.last_hidden_state.squeeze().size()
#print(outputembedding)

cpu


In [4]:
# TODO: Fill out ReviewDataset
class ReviewDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe["title"]+" "+dataframe["sentence"]

        self.max_len = max_len

        self.targets = dataframe["score"]


    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

        #return {
        #    'ids': torch.tensor(ids, dtype=torch.long),
        #    'mask': torch.tensor(mask, dtype=torch.long),
        #    'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)}

In [6]:
class ReviewDataset1(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe["title"]+" "+dataframe["sentence"]

        self.max_len = max_len

        #self.targets = dataframe["score"]


    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

        #return {
        #    'ids': torch.tensor(ids, dtype=torch.long),
        #    'mask': torch.tensor(mask, dtype=torch.long),
        #    'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)}

In [ ]:
with torch.no_grad():
  train_dataset = ReviewDataset(train_val,tokenizer,MAX_LEN)
  test_dataset = ReviewDataset1(test_val,tokenizer,MAX_LEN)


  train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=4, pin_memory=True)
  test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=4, pin_memory=True)
# Additional code if needed


In [8]:
# TODO: Fill out MyModule
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        #self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
    #def forward(self,preprocess):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = F.sigmoid(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output=10*F.sigmoid(output)
        return output


model = MyModule().to(device)

In [ ]:
# TODO: Setup loss function, optimiser, and scheduler

optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = nn.MSELoss()
#optim = torch.optim.SGD(model.parameters(), lr=0.001,momentum=0.9)
scheduler = None


for epoch in range(NUM_EPOCHS):
    print(f'Current epoche:{epoch}')
    run_loss = 0
    model.train()

    for i,data in tqdm(enumerate(train_loader,0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        #model_frozen=DistilBertModel.from_pretrained("distilbert-base-uncased")
        #preprocess = model_frozen(input_ids=ids, attention_mask=mask)

        outputs = model(ids, mask, token_type_ids)
        #loss = outputs.loss
        optimizer.zero_grad()
        loss=criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        #scheduler.step()
        run_loss = run_loss + loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {run_loss / 200:.3f}')
            run_loss = 0
        #i=i+1
        # TODO: Set up training loop

In [ ]:
model.eval()

results=[]
with torch.no_grad():
    for _, data in tqdm(enumerate(testing_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        #targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)
        outputs[outputs>10]=10
        outputs[outputs<0]=0
        #fin_targets.extend(targets.cpu().detach().numpy().tolist())
        results.extend(outputs.cpu().detach().numpy().tolist())
    with open("result.txt", "w") as f:
        for val in np.concatenate(results):
            f.write(f"{val}\n")

NameError: name 'model' is not defined